In [7]:
import numpy as np
import pandas as pd
import folium
import branca
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import geojsoncontour
import base64
from folium.features import DivIcon
from folium import plugins
import json, ast
import scipy as sp
import scipy.ndimage

from pykrige.ok import OrdinaryKriging
from pykrige.kriging_tools import write_asc_grid
import pykrige.kriging_tools as kt

from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [16]:
def color_producer1(el):
    if el < 0.01:
        return "#FFFFFF"
    elif 0.01 <= el < 0.1:
        return "#BBBBFF"
    elif 0.11 <= el < 0.2:
        return "#8888FF"
    elif 0.21 <= el < 0.6:
        return "#4444FF"

    else:
        return "#0000FF"

def color_producer2(el):
    if el < 0.01:
        return "#FFFFFF"
    elif 0.01 <= el < 0.1:
        return "#FFBBBB"
    elif 0.11 <= el < 0.2:
        return "#FF8888"
    elif 0.21 <= el < 0.6:
        return "#FF4444"

    else:
        return "#FF0000"
def add_layer(df,text,cc): 
    if cc == 1:
        colors = ["#FFFFFF", "#6666FF", "#6666FF","#6666FF", "#6666FF","#6666FF",
          "#0000FF","#0000FF","#0000FF","#0000FF"]
    elif cc == 2:
        colors = colors = ["#FFFFFF", "#FFAAAA", "#FF6666","#FF6666", "#FF6666","#FF6666",
          "#FF0000","#FF0000","#FF0000","#FF0000"] 

    polygon = Polygon([
    [120.7844750, 21.8371610],
    [120.9053645, 22.0002358],
    [120.8943745, 22.3054952],
    [121.0042741, 22.5593698],
    [121.2020934, 22.7114709],
    [121.4328825, 23.1162462],
    [121.6307017, 23.9020427],
    [121.6746615, 24.0626488],
    [121.8285209, 24.4232770],
    [121.8834707, 24.5532518],
    [121.8505009, 24.7928514],
    [121.9933703, 24.9622885],
    [121.6087218, 25.2905298],
    [121.3119929, 25.1314927],
    [121.0042741, 25.0419431],
    [120.7954649, 24.6431556],
    [120.1140876, 23.7211221],
    [120.0151779, 23.0151658],
    [120.3119068, 22.4680284],
    [120.6416055, 22.3359846],
    [120.7844750, 21.8371610]
    ])
    lons = np.asarray(df.long.tolist())
    lats = np.asarray(df.lat.tolist())
    data = np.asarray(df.EI.tolist())
    grid_space = 0.05
    global grid_lon, grid_lat
    grid_lon = np.arange(lons.min()-0.05, lons.max()+0.1, grid_space) 
    grid_lat = np.arange(lats.min()-0.05, lats.max()+0.1, grid_space)
    OK = OrdinaryKriging(lons, lats, data, variogram_model='gaussian', verbose=False, enable_plotting=False,nlags=20)
    global z_mesh,x_mesh,y_mesh
    z_mesh, ss1 = OK.execute('grid', grid_lon, grid_lat)
    x_mesh, y_mesh = np.meshgrid(grid_lon, grid_lat)
    shp = x_mesh.shape
    mtw = np.zeros(shp,dtype=np.float)
    for i in range(shp[0]):
        for j in range(shp[1]):
            point = Point(x_mesh[i][j], y_mesh[i][j])
            if not polygon.contains(point):
                mtw[i][j] = 1        
    z_mesh = np.ma.masked_where(mtw == 1, z_mesh)    
    
    contourf = plt.contourf(x_mesh, y_mesh, z_mesh, levels, alpha=0.8, colors=colors, linestyles='None', vmin=vmin, vmax=vmax)
    global geojson
    geojson = geojsoncontour.contourf_to_geojson(
        contourf=contourf,
        min_angle_deg=3.0,
        ndigits=5,
        stroke_width=1,
        fill_opacity=0.5)
    geoj = folium.GeoJson(
        geojson,
        style_function=lambda x: {
            'color':     x['properties']['stroke'],
            'weight':    x['properties']['stroke-width'],
            'fillColor': x['properties']['fill'],
            'opacity':   0.6,
        })
#    f1.add_child(geoj)

    return geoj,geojson

def add_point(df,text,cc):
    html = """
    CWB station id:
    <br><a href="https://www.google.com/search?q=%%22%s%%22" target="_blank">%s</a><br>
    EI value: %.2f 
    """
    
#    f2 = folium.FeatureGroup(name='{}'.format(text))
    for lt, ln, ei, nm in zip(df.lat,df.long, df.EI, df.id):
        iframe = folium.IFrame(html=html % (nm,nm,ei), width = 200, height = 100)
        if cc == 1:
            p = folium.CircleMarker(location = [lt,ln], 
                                        popup = folium.Popup(iframe),
                                        radius = 4,
                                        color = 'grey',
                                        fill_color=color_producer1(ei),
                                        fill_opacity = 0.2)
        elif cc == 2:
            p = folium.CircleMarker(location = [lt,ln], 
                                        popup = folium.Popup(iframe),
                                        radius = 4,
                                        color = 'grey',
                                        fill_color=color_producer2(ei),
                                        fill_opacity = 0.2)
    return p

def add_text(loc,content,c,fig):
    html = '<img src="data:image/png;base64,{}">'.format
    encoded = base64.b64encode(open("../image/{}.jpg".format(fig), 'rb').read()).decode()

    iframe = folium.IFrame(html(encoded), width=200, height=200)
    popup = folium.Popup(iframe, max_width=1000)

    Text = folium.Marker(location=loc, popup=popup,
                         icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<font color="{}"><div style="font-size: 16pt">{}</div></font>'.format(c,content),
            ))
    return Text

In [17]:
def add_dictime(geojson,time):
    t = []
    for i,ply in enumerate(ast.literal_eval(geojson)['features']):
        dic = {}
        dic['geometry'] = ply['geometry']
        dic['properties'] = {"times" : ["{}".format(time)], "style": 
                             {"color":ply['properties']['fill'],
                            "opacity" :0,"weight" : 0.5, "fillColor": ply['properties']['fill']
                             }}
        dic['type'] = 'Feature'
        t.append(dic)
    return t

In [10]:
sp1, sp2 = 'orius_strigicollis','cacopsylla_chinensis'
df1 = pd.read_csv("../output_csv/combine/{}.csv".format(sp1))
df2 = pd.read_csv("../output_csv/combine/{}.csv".format(sp2))
df1 = df1[df1.long>120]
df1 = df1[df1['id'] != 'C0S730'][df1['id'] != '467620'][df1['id'] != '466950'][df1['id'] !='C0W170']
df2 = df2[df2.long>120]
df2 = df2[df2['id'] != 'C0S730'][df2['id'] != '467620'][df2['id'] != '466950'][df2['id'] !='C0W170']

/Users/nghianc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/Users/nghianc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [18]:
tl = []
levels,vmin, vmax = 5,0,1
for time in list(set(df1.Time)):
    dft = df1[df1.Time == time]
    _,geojson = add_layer(dft,'',2)
    t = add_dictime(geojson,time)
    tl += t

In [19]:
geomap = folium.Map([23.75, 121], zoom_start=8, tiles="cartodbpositron")
colors = ["#FFFFFF", "#FFAAAA", "#FF6666","#FF6666", "#FF6666","#FF6666",
          "#FF0000","#FF0000","#FF0000","#FF0000"] 
cm     = branca.colormap.LinearColormap(colors, vmin=0, vmax=1).to_step(len(colors))
cm.caption = 'EI value'
geomap.add_child(cm)
plugins.TimestampedGeoJson(
    {
        'type': 'Feature',
        'features': tl,
        
    },
    period='P1M',
    duration='P15D',
    auto_play=False,
    loop=False,
    loop_button=True,
    date_options='YYYY/MM',
).add_to(geomap)
#geomap
plugins.Fullscreen(position='topright', force_separate_button=True).add_to(geomap)
geomap.save('../html/test.html')

In [ ]:
# geomap = folium.Map([23.75, 121], zoom_start=8, tiles="cartodbpositron")
# levels = 5
# vmin, vmax = 0,1.01
# f1 = folium.FeatureGroup(name='test1')
# geoj1,geojson1 = add_layer(df1,"1",1)
# #p1 = add_point(df1,"1",1)
# #Text1 = add_text([25,119],"南方小花蝽","blue","orius_strigicollis")
# #geomap.add_child(Text1)
# f2 = folium.FeatureGroup(name='test2')
# geoj2,_ = add_layer(df2,"1",2)
# #p2 = add_point(df2,"1",2)
# #Text2 = add_text([24,119],"中國梨木蝨","red","cacopsylla_chinensis")
# #geomap.add_child(Text2)

# #Text2 = add_text([25,123],"2017-06","black","cacopsylla_chinensis")
# #geomap.add_child(Text2)
# f1.add_child(geoj1)
# f2.add_child(geoj2)
# #f1.add_child(p1)
# #f2.add_child(p2)
# geomap.add_child(f1)
# geomap.add_child(f2)

# plugins.Fullscreen(position='topright', force_separate_button=True).add_to(geomap)
# geomap.add_child(folium.LayerControl()) 
# #geomap.save('../html/biozone2017-06.html')
# geomap